<a href="https://colab.research.google.com/github/cshum/shopline-utils/blob/main/shopline_import_multiplier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Shopline Import Multiplier

In [ ]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_columns', None)  
#pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 999)

#input/output config
xls_file = 'http://static.cshum.com.s3.amazonaws.com/shopline-utils/ProductBulkImportForm-2021-04-21-21_30.xls'
image_filenames = 'http://static.cshum.com.s3.amazonaws.com/shopline-utils/1.txt'
output_filename = 'shopline.xls'

#image url: substitute template where suitable
image_url_template = 'http://static.cshum.com.s3.amazonaws.com/shopline-utils/images/{filename}'
image_url_substitute = '{filename}'


group_field = 'Product Handle*'
sku_field = 'Variation SKU'
color_field = 'Variation name A(English)'
image_field = 'Images*'


def process(df):
  #skip first row
  df_first_row = df.loc[0].to_frame().T
  df = df.iloc[1:].reset_index(drop=True).ffill()
  #cleanup existing duped products
  df = df.drop_duplicates(sku_field) 
  #group by product handle for processing
  df = df.groupby(group_field).apply(process_group).reset_index(drop=True)
  #restore first row
  df = pd.concat([df_first_row, df]).reset_index(drop=True)
  return df


def process_group(df):
  df = df.copy()
  ln = df.shape[0]
  l = df.index.to_list()
  if ln <= 1:
    return df
  first_color = df.loc[0][color_field]
  for i in range(1, ln):
    #contine next row if color same as first color
    if first_color == df.loc[l[i]][color_field]:
      continue
    #duplicate variants a,b,c then b,c,a then c,a,b
    ps = l[i:] + l[0:i]
    for idx in ps:
      p = df.loc[idx].copy()
      # append 002 003... to product handle to group duplications
      p[group_field] = '{}{:0>3d}'.format(p[group_field], i+1)
      df.loc[df.shape[0] + 1] = p
  return df


def process_merge_images(df, df_images):
  #excel left join images from images
  df = df.merge(df_images, on=sku_field, how='left')
  df[image_field] = df['images'].fillna(df[image_field])
  del df['images']
  return df


def process_filenames(df):
  df = df.copy()
  #extract sku from filename excluding last 2 digits and file extensions
  df[sku_field] = df.filename.str.extract('(.+)\d{2}\.[a-zA-Z]+$', expand=True)
  #extract sequence from filename last 2 digits excluding file extensions
  df['seq'] = df.filename.str.extract('(\d{2})\.[a-zA-Z]+$', expand=True).astype(int)
  #form image url
  df['url'] = df['filename'].apply(
    lambda u: image_url_template.replace(image_url_substitute, u))
  join_urls = lambda df: ','.join(df.sort_values(by='seq')['url'].to_list())
  #return comma joined image urls group by sku
  return df.groupby(sku_field).apply(join_urls).reset_index(name='images')


df = pd.read_excel(xls_file)
df_images = process_filenames(pd.read_csv(image_filenames, names=['filename']))
df = process_merge_images(df, df_images)
df = process(df)
df.to_excel(output_filename, index=False)
df